In [4]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB

In [6]:
df = pd.read_csv('data/med_inv_dataset.csv') # For local

df.columns = df.columns.str.lower()
df = df.dropna()
df['dateofbill'] = pd.to_datetime(df['dateofbill'])
df['month_name'] = df['dateofbill'].dt.strftime('%B') # Extract month name
df['month_number'] = df['dateofbill'].dt.month  # Extract month number
df['week_number'] = df['dateofbill'].dt.isocalendar().week  # Extract week number
df.sample(5)

,typeofsales,patient_id,specialisation,dept,dateofbill,quantity,returnquantity,final_cost,final_sales,rtnmrp,formulation,drugname,subcat,subcat1,month_name,month_number,week_number
11735,Sale,12018079461,Specialisation7,Department1,2022-09-02,1,0,46.676,47.584,0.0,Form1,SODIUM CHLORIDE IVF 100ML,"IV FLUIDS, ELECTROLYTES, TPN",INTRAVENOUS & OTHER STERILE SOLUTIONS,September,9,35
10227,Sale,12018107236,Specialisation16,Department1,2022-08-14,2,0,47.754,104.204,0.0,Form1,CEFTRIAXONE 1GM,INJECTIONS,ANTI-INFECTIVES,August,8,32
8441,Sale,12018076747,Specialisation4,Department1,2022-04-15,1,0,95.020,106.760,0.0,Form1,MYCOPHENOLATE MOFETIL 250MG CAP,TABLETS & CAPSULES,IMMUNOLOGY,April,4,15
1870,Sale,12018095967,Specialisation7,Department1,2022-06-05,1,0,62.494,71.000,0.0,Form1,ANHYDROUS DEXTROSE 1000MG + ASCORBIC ACID 500M...,INJECTIONS,VITAMINS & MINERALS,June,6,22
7622,Sale,12018068793,Specialisation33,Department2,2022-08-03,1,0,44.368,45.500,0.0,Form1,BUPIVACAINE 5MG,INJECTIONS,ANAESTHETICS,August,8,31


In [7]:
# Create a bi-weekly period column
df['bi_weekly'] = (df['dateofbill'].dt.day - 1) // 14 + 1

# Group by drug, subcat, month_name, month_number, and bi-weekly period
df_bi_weekly = df.groupby(['drugname', 'subcat', 'month_name', 'month_number', 'bi_weekly'], as_index=False).agg(
    {
        'quantity': 'sum',
        'returnquantity': 'sum',
        'final_cost': 'sum',
        'final_sales': 'sum',
        'rtnmrp': 'sum'
    }
)

df_bi_weekly.head()

,drugname,subcat,month_name,month_number,bi_weekly,quantity,returnquantity,final_cost,final_sales,rtnmrp
0,ACEBROPHYLLINE 100MG CAP,TABLETS & CAPSULES,August,8,1,1,0,64.622,72.40,0.0
1,ACEBROPHYLLINE 100MG CAP,TABLETS & CAPSULES,May,5,2,1,0,62.766,69.64,0.0
2,ACEBROPHYLLINE 200MG TAB,TABLETS & CAPSULES,April,4,3,0,1,76.800,0.00,86.0
3,ACEBROPHYLLINE 200MG TAB,TABLETS & CAPSULES,August,8,2,2,0,149.924,172.00,0.0
4,ACEBROPHYLLINE 200MG TAB,TABLETS & CAPSULES,December,12,1,1,0,74.962,86.00,0.0


In [8]:
# Step 1: Collect top 5 subcategories with the highest sum of quantity
top_5_subcats = df_bi_weekly.groupby('subcat')['quantity'].sum().nlargest(5).index

# Step 2: Filter the dataframe for only the top 5 subcategories
filtered_top_5_per_subcat = df_bi_weekly[df_bi_weekly['subcat'].isin(top_5_subcats)]

# Step 3: Aggregate quantity sum grouping by subcat and drugname
top_5_drugs_per_subcat = (
    filtered_top_5_per_subcat.groupby(['subcat', 'drugname'])['quantity']
    .sum()
    .reset_index()
    .sort_values(['subcat', 'quantity'], ascending=[True, False])
    .groupby('subcat')
    .head(5)
)

top_5_drugs_per_subcat

,subcat,drugname,quantity
9,INHALERS & RESPULES,LEVOSALBUTAMOL/LEVALBUTEROL 0.63MG RESPULES,604
11,INHALERS & RESPULES,SALBUTAMOL 2.5MG,285
2,INHALERS & RESPULES,BUDESONIDE 2ML,169
6,INHALERS & RESPULES,LEVOSALBUTAMOL 1.25MG + IPRATROPIUM BROMIDE 50...,169
7,INHALERS & RESPULES,LEVOSALBUTAMOL 1.25MG RESPULES,85
185,INJECTIONS,ONDANSETRON 2MG/ML,933
243,INJECTIONS,WATER FOR INJECTION 10ML SOLUTION,871
192,INJECTIONS,PANTOPRAZOLE 40MG INJ,594
195,INJECTIONS,PARACETAMOL 1GM IV INJ,593
182,INJECTIONS,NORADRENALINE 2ML INJ,553


In [9]:
filtered_df_bi_weekly = df_bi_weekly.merge(top_5_drugs_per_subcat[['subcat', 'drugname']], on=['subcat', 'drugname'])
filtered_df_bi_weekly = filtered_df_bi_weekly.sort_values(by=['subcat', 'drugname', 'month_number', 'bi_weekly'])
# Add a biweekly index for every drugname in every subcat
filtered_df_bi_weekly['biweekly_index'] = (
    filtered_df_bi_weekly.groupby(['subcat', 'drugname'])
    .cumcount() + 1
)
filtered_df_bi_weekly

,drugname,subcat,month_name,month_number,bi_weekly,quantity,returnquantity,final_cost,final_sales,rtnmrp,biweekly_index
11,BUDESONIDE 2ML,INHALERS & RESPULES,January,1,1,6,0,142.856,268.668,0.000,1
12,BUDESONIDE 2ML,INHALERS & RESPULES,January,1,2,15,0,217.322,671.670,0.000,2
13,BUDESONIDE 2ML,INHALERS & RESPULES,January,1,3,12,0,165.858,537.336,0.000,3
9,BUDESONIDE 2ML,INHALERS & RESPULES,February,2,1,6,2,190.470,270.210,90.584,4
10,BUDESONIDE 2ML,INHALERS & RESPULES,February,2,2,3,0,91.464,134.334,0.000,5
...,...,...,...,...,...,...,...,...,...,...,...
631,VALGANCICLOVIR 450MG TAB,TABLETS & CAPSULES,September,9,1,6,0,748.114,1039.992,0.000,14
632,VALGANCICLOVIR 450MG TAB,TABLETS & CAPSULES,September,9,2,4,0,485.410,693.328,0.000,15
630,VALGANCICLOVIR 450MG TAB,TABLETS & CAPSULES,October,10,1,9,0,1082.174,1559.988,0.000,16
628,VALGANCICLOVIR 450MG TAB,TABLETS & CAPSULES,November,11,1,11,1,1456.230,1906.652,173.332,17


In [10]:
subcat_drug_dict = top_5_drugs_per_subcat.groupby('subcat')['drugname'].apply(list).to_dict()
subcat_drug_dict

{'INHALERS & RESPULES': ['LEVOSALBUTAMOL/LEVALBUTEROL 0.63MG RESPULES',
  'SALBUTAMOL 2.5MG',
  'BUDESONIDE 2ML',
  'LEVOSALBUTAMOL 1.25MG + IPRATROPIUM BROMIDE 500MCG RESPULES',
  'LEVOSALBUTAMOL 1.25MG RESPULES'],
 'INJECTIONS': ['ONDANSETRON 2MG/ML',
  'WATER FOR INJECTION 10ML SOLUTION',
  'PANTOPRAZOLE 40MG INJ',
  'PARACETAMOL 1GM IV INJ',
  'NORADRENALINE 2ML INJ'],
 'IV FLUIDS, ELECTROLYTES, TPN': ['SODIUM CHLORIDE IVF 100ML',
  'SODIUM CHLORIDE 0.9%',
  'MULTIPLE ELECTROLYTES 500ML IVF',
  'SODIUM CHLORIDE 0.9% IVF 1000ML',
  'HUMAN ALBUMIN 25% INJ'],
 'LIQUIDS & SOLUTIONS': ['SEVOFLURANE 99.97%',
  'SEVOFLURANE',
  'CHLORHEXIDINE GLUCONATE 0.2%W/V',
  'SODIUM PHOSPHATE 1.963GM POWDER',
  'LACTULOSE 300ML SYP'],
 'TABLETS & CAPSULES': ['VALGANCICLOVIR 450MG TAB',
  'URSODEOXYCHOLIC ACID 300MG TAB',
  'TACROLIMUS 1MG CAP',
  'ESOMEPRAZOLE 40MG',
  'PANTOPRAZOLE 40MG TAB']}

In [ ]:
def calculate_last_three_cycles(df, subcat, drugname, horizon = 'biweekly_index', quanity= 'quantity', all_filter = False, code = False):
  ml_df = df[(df['subcat'] == subcat) & (df['drugname'] == drugname)][[horizon,'drugname', 'subcat', quanity]]

  ml_df['quantity_lastcycle']=ml_df[quanity].shift(+1)
  ml_df['quantity_2cycleback']=ml_df[quanity].shift(+2)
  ml_df['quantity_3cycleback']=ml_df[quanity].shift(+3)
  ml_df['quantity_4cycleback']=ml_df[quanity].shift(+4)
  ml_df['quantity_5cycleback']=ml_df[quanity].shift(+5)

  ml_df = ml_df.dropna() #dropping na is necessary to avoid model failure other option

  X = ml_df[['subcat','drugname', horizon,'quantity_lastcycle', 'quantity_2cycleback', 'quantity_3cycleback', 'quantity_4cycleback', 'quantity_5cycleback']]
  y = ml_df[quanity]
  return X, y

In [20]:
# Filter rows where month_number is 4, 5, or 6
filtered_months_df = filtered_df_bi_weekly[filtered_df_bi_weekly['month_number'].isin([4, 5, 6])]

# The rest of the DataFrame
rest_df = filtered_df_bi_weekly[~filtered_df_bi_weekly['month_number'].isin([4, 5, 6])]

In [52]:
filtered_months_df.shape

(156, 11)

In [53]:
rest_df.shape

(509, 11)

In [44]:
trainX = pd.DataFrame(columns=['subcat', 'drugname','quantity_lastcycle', 'quantity_2cycleback', 'quantity_3cycleback',
                              'quantity_4cycleback','quantity_5cycleback'])
for key in subcat_drug_dict.keys():
    for drug in subcat_drug_dict[key]:
        # print(key, drug)
        X, y = calculate_last_three_cycles(rest_df, subcat = key, drugname=drug, quanity = 'quantity', all_filter = False, code = False)
        trainX = pd.concat([X, trainX])

/var/folders/dc/n7cp43k94blc5lmsv8p725pc0000gn/T/ipykernel_14606/3078485149.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trainX = pd.concat([X, trainX])


In [46]:
trainX

,subcat,drugname,biweekly_index,quantity_lastcycle,quantity_2cycleback,quantity_3cycleback,quantity_4cycleback,quantity_5cycleback
349,TABLETS & CAPSULES,PANTOPRAZOLE 40MG TAB,6.0,3.0,1.0,1.0,5.0,2.0
344,TABLETS & CAPSULES,PANTOPRAZOLE 40MG TAB,10.0,1.0,3.0,1.0,1.0,5.0
336,TABLETS & CAPSULES,PANTOPRAZOLE 40MG TAB,11.0,4.0,1.0,3.0,1.0,1.0
337,TABLETS & CAPSULES,PANTOPRAZOLE 40MG TAB,12.0,2.0,4.0,1.0,3.0,1.0
355,TABLETS & CAPSULES,PANTOPRAZOLE 40MG TAB,13.0,1.0,2.0,4.0,1.0,3.0
...,...,...,...,...,...,...,...,...
197,INHALERS & RESPULES,LEVOSALBUTAMOL/LEVALBUTEROL 0.63MG RESPULES,27.0,49.0,12.0,14.0,10.0,5.0
198,INHALERS & RESPULES,LEVOSALBUTAMOL/LEVALBUTEROL 0.63MG RESPULES,28.0,27.0,49.0,12.0,14.0,10.0
179,INHALERS & RESPULES,LEVOSALBUTAMOL/LEVALBUTEROL 0.63MG RESPULES,29.0,11.0,27.0,49.0,12.0,14.0
180,INHALERS & RESPULES,LEVOSALBUTAMOL/LEVALBUTEROL 0.63MG RESPULES,30.0,0.0,11.0,27.0,49.0,12.0


In [47]:
testX = pd.DataFrame(columns=['subcat', 'drugname','quantity_lastcycle', 'quantity_2cycleback', 'quantity_3cycleback',
                              'quantity_4cycleback','quantity_5cycleback'])
for key in subcat_drug_dict.keys():
    for drug in subcat_drug_dict[key]:
        # print(key, drug)
        X, y = calculate_last_three_cycles(filtered_months_df, subcat = key, drugname=drug, quanity = 'quantity', all_filter = False, code = False)
        testX = pd.concat([X, testX])
        # break
    # break

/var/folders/dc/n7cp43k94blc5lmsv8p725pc0000gn/T/ipykernel_14606/1020696423.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  testX = pd.concat([X, testX])


In [51]:
testX = testX.sort_values(by='biweekly_index')
testX

,subcat,drugname,biweekly_index,quantity_lastcycle,quantity_2cycleback,quantity_3cycleback,quantity_4cycleback,quantity_5cycleback
146,INHALERS & RESPULES,LEVOSALBUTAMOL 1.25MG + IPRATROPIUM BROMIDE 50...,12.0,4.0,9.0,4.0,5.0,12.0
404,INHALERS & RESPULES,SALBUTAMOL 2.5MG,12.0,3.0,14.0,1.0,2.0,8.0
604,TABLETS & CAPSULES,URSODEOXYCHOLIC ACID 300MG TAB,13.0,3.0,3.0,2.0,2.0,3.0
497,"IV FLUIDS, ELECTROLYTES, TPN",SODIUM CHLORIDE 0.9% IVF 1000ML,13.0,15.0,3.0,11.0,8.0,9.0
254,INJECTIONS,NORADRENALINE 2ML INJ,13.0,13.0,3.0,30.0,10.0,20.0
191,INHALERS & RESPULES,LEVOSALBUTAMOL/LEVALBUTEROL 0.63MG RESPULES,13.0,17.0,28.0,25.0,5.0,29.0
556,LIQUIDS & SOLUTIONS,SODIUM PHOSPHATE 1.963GM POWDER,13.0,1.0,3.0,2.0,3.0,3.0
103,"IV FLUIDS, ELECTROLYTES, TPN",HUMAN ALBUMIN 25% INJ,13.0,10.0,5.0,2.0,6.0,13.0
17,INHALERS & RESPULES,BUDESONIDE 2ML,14.0,14.0,4.0,3.0,1.0,6.0
292,INJECTIONS,ONDANSETRON 2MG/ML,14.0,23.0,31.0,1.0,28.0,33.0


In [2]:
df.head()

,Day,Drug,Predicted_Demand,Unit_Cost,Salvage_Value,Return_Prediction,Shelf_Life
0,1,Drug A,43,33.690086,6.971516,5,11
1,2,Drug A,33,44.924573,1.045554,6,13
2,3,Drug A,19,26.249672,2.447272,2,11
3,4,Drug A,47,10.381741,5.938604,9,9
4,5,Drug A,12,37.096015,7.227057,2,14


## Final Consolidation

In [ ]:
# Forecasted demand from Random Forest (example data for 30 days)
np.random.seed(42)
T = 30  # Planning horizon
drugs = ["Drug A", "Drug B", "Drug C", "Drug D"]

df = pd.DataFrame({
    "Day": np.tile(np.arange(1, T+1), len(drugs)),
    "Drug": np.repeat(drugs, T),
    "Predicted_Demand": np.random.randint(5, 50, T * len(drugs)),
    "Unit_Cost": np.random.uniform(5, 50, T * len(drugs)),  
    "Salvage_Value": np.random.uniform(1, 10, T * len(drugs)),  
    "Return_Prediction": np.random.randint(0, 10, T * len(drugs)),  
    "Shelf_Life": np.random.randint(5, 15, T * len(drugs))  
})

In [57]:
# Fix values per drug (same across all days)
fixed_values = {
    "Drug A": {"Unit_Cost": 20, "Salvage_Value": 5, "Shelf_Life": 10},
    "Drug B": {"Unit_Cost": 25, "Salvage_Value": 6, "Shelf_Life": 12},
    "Drug C": {"Unit_Cost": 30, "Salvage_Value": 7, "Shelf_Life": 15},
    "Drug D": {"Unit_Cost": 18, "Salvage_Value": 4, "Shelf_Life": 8},
}

# Apply fixed values to each row based on drug type
df["Unit_Cost"] = df["Drug"].map(lambda x: fixed_values[x]["Unit_Cost"])
df["Salvage_Value"] = df["Drug"].map(lambda x: fixed_values[x]["Salvage_Value"])
df["Shelf_Life"] = df["Drug"].map(lambda x: fixed_values[x]["Shelf_Life"])

In [73]:
df

,Day,Drug,Predicted_Demand,Unit_Cost,Salvage_Value,Return_Prediction,Shelf_Life
0,1,Drug A,43,33.690086,6.971516,5,11
1,2,Drug A,33,44.924573,1.045554,6,13
2,3,Drug A,19,26.249672,2.447272,2,11
3,4,Drug A,47,10.381741,5.938604,9,9
4,5,Drug A,12,37.096015,7.227057,2,14
...,...,...,...,...,...,...,...
115,26,Drug D,27,12.273292,6.941776,8,12
116,27,Drug D,28,45.434938,3.519405,3,5
117,28,Drug D,41,32.289308,9.593788,4,10
118,29,Drug D,39,5.413867,7.641072,3,5


## Optimization Model

In [86]:
# Gurobi Model
model = Model("Multi_Period_Medical_Inventory_Optimization")

# Decision Variables
Q = model.addVars(df.index, vtype=GRB.CONTINUOUS, name="OrderQty")  # Order quantity

# State Variables
I = model.addVars(df.index, vtype=GRB.CONTINUOUS, name="Inventory")  # Inventory level

# Auxiliary Variables (Derived)
Y = model.addVars(df.index, vtype=GRB.CONTINUOUS, name="Expired")   # Expired stock
S = model.addVars(df.index, vtype=GRB.CONTINUOUS, name="Stockout")  # Stockout

# Cost Coefficients
holding_cost = 2  # Cost per unit held
stockout_penalty = 50  # Cost per stockout
waste_penalty = 10  # Cost for expired stock

# Objective: Minimize Total Cost
model.setObjective(
    sum(df.loc[i, "Unit_Cost"] * Q[i] + 
        holding_cost * (I[i]) +
        stockout_penalty * S[i] + 
        waste_penalty * Y[i] - 
        df.loc[i, "Salvage_Value"] * df.loc[i, "Return_Prediction"]
        for i in df.index),
    GRB.MINIMIZE
)

In [87]:
# Constraints:
for i in df.index:
    # Inventory Balance Constraint
    if i >= T:  # Ensure we don't reference out-of-bounds indices
        continue
    model.addConstr(I[i+1] == I[i] + Q[i] + df.loc[i, "Return_Prediction"] - df.loc[i, "Predicted_Demand"] - Y[i], name=f"Inventory_Balance_{i}")

    model.addConstr(I[i+1] <= 100, name=f"Space_constraint{i}")
    
    # Expired Inventory Constraint
    model.addConstr(Y[i] <= I[i], name=f"Expiry_{i}")

    # Stockout Constraint
    model.addConstr(S[i] >= df.loc[i, "Predicted_Demand"] - I[i] - Q[i], name=f"Stockout_{i}")

In [88]:
# Solve Model
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 24.3.0 24D81)

CPU model: Intel(R) Core(TM) i7-8557U CPU @ 1.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 120 rows, 480 columns and 300 nonzeros
Model fingerprint: 0xad4a45e8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 32 rows and 362 columns
Presolve time: 0.01s
Presolved: 88 rows, 118 columns, 263 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.9063568e+03   1.848750e+02   0.000000e+00      0s
      56    7.2073415e+03   0.000000e+00   0.000000e+00      0s

Solved in 56 iterations and 0.03 seconds (0.00 work units)
Optimal objective  7.207341522e+03


In [89]:
# Store Results
df["Optimal_Order"] = [Q[i].x for i in df.index]
df["Inventory_Level"] = [I[i].x for i in df.index]
df["Expired_Stock"] = [Y[i].x for i in df.index]
df["Stockouts"] = [S[i].x for i in df.index]

In [90]:
df.head()

,Day,Drug,Predicted_Demand,Unit_Cost,Salvage_Value,Return_Prediction,Shelf_Life,Optimal_Order,Inventory_Level,Expired_Stock,Stockouts
0,1,Drug A,43,33.690086,6.971516,5,11,0.0,138.0,0.0,0.0
1,2,Drug A,33,44.924573,1.045554,6,13,0.0,100.0,0.0,0.0
2,3,Drug A,19,26.249672,2.447272,2,11,0.0,73.0,0.0,0.0
3,4,Drug A,47,10.381741,5.938604,9,9,82.0,56.0,0.0,0.0
4,5,Drug A,12,37.096015,7.227057,2,14,0.0,100.0,0.0,0.0
